### Baseline model (Plus)에 있는 기능

- binary_load_dataset 메서드

데이터 비율 조정을 위한 파라미터 값을 기반으로 데이터를 로드하는 메서드로

주어진 binary_classification_data.csv 파일을 열고 데이터를 읽습니다. 데이터의 마지막 열에 종속 변수가 있으며, 해당 열의 값에 따라서 '1'인 pulsar 데이터와 '0'인 star 데이터를 각각 별도의 리스트에 저장합니다.

그 후, 실험에 사용될 데이터와 신경망의 입력 및 출력 크기를 전역 변수로 선언합니다. pulsar와 star 데이터는 하나의 변수 data에 그룹화됩니다. pulsar 데이터의 수를 star 데이터의 수에 맞추기 위해 데이터의 수를 조정할 수 있습니다.

adjust_ratio 매개변수가 True인 경우, pulsar 데이터의 비율을 증가시킵니다. pulsar 데이터와 star 데이터를 하나의 변수에 할당하고, 데이터의 크기는 star 데이터의 배수로 저장됩니다.

adjust_ratio 값이 True인 경우 star 데이터를 data 변수에 할당한 후, pulsar 데이터를 star 데이터의 수만큼 반복하여 추가합니다. 이를 위해 % 연산자를 사용하여 pulsar 데이터의 인덱스를 순차적으로 추출합니다.

adjust_ratio 값이 False인 경우, 기존 데이터를 그대로 data 변수에 할당합니다. star 데이터를 먼저 할당하고, 그 다음에 pulsar 데이터를 할당합니다.

### 데이터 불러오기

In [7]:
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

In [3]:
df = pd.read_csv('binary_classification_data.csv')
target = 'target_class'
y = df[target]
x = df.drop(target, axis = 1)

### 데이터 불균형 처리
- 데이터 처리

1. oversampling :

   SMOTE, ADASYN

2. undersampling

- 평가지표 처리

1. accurcacy :

   대부분의 예측이 다수 클래스에 속하게 되어 별로

2. F1-score :

   Baseline model에서 사용한 평가지표로 불균형한 데이터에서 평가지표로 사용

- 모델 구현시 처리

1. 클래스 가중치 설정 :

   소수 클래스에 더 큰 가중치 부여


In [4]:
# 유저로 부터 입력을 받아 검증 데이터 셋을 사용할 것인지, 표준화를 사용할 것인지 정함.
print("[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요")
input_1 = input("[안내] 데이터를 표준화 하시겠습니까? : ")
# 표준화 진행 여부
if input_1 == 'y':
    scaler = StandardScaler()
    X = scaler.fit_transform(x)
    print("[안내] 데이터 표준화를 진행했습니다.")
else:
    X = x
    print("[안내] 데이터 표준화를 진행하지 않습니다.")
    
input_2 = input("[안내] 검증 데이터셋을 분리할까요? : ")



# 검증 데이터 진행 여부
if input_2 == 'y':
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    print("[안내] 검증 데이터를 추가로 분리했습니다.")

else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("[안내] 검증 데이터를 분리하지 않았습니다.")



[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요
[안내] 데이터 표준화를 진행했습니다.
[안내] 검증 데이터를 추가로 분리했습니다.


In [8]:
# SMOTE로 oversampling 진행
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 클래스 가중치 계산
class_weight = {0: 1., 1: (len(y_train) / sum(y_train))}

### 모델 구현 - 작업중
- 최적의 파라미터를 반복문을 통해 찾기

In [10]:
# 모델 구현
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_resampled.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, class_weight=class_weight)

# 모델 평가
y_pred = model.predict(X_test)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
f1 = f1_score(y_test, y_pred_binary)
roc_auc = roc_auc_score(y_test, y_pred)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
loss, acc = model.evaluate(X_val, y_val, verbose = 2)

print('F1-score:', f1)
print('AUC-ROC:', roc_auc)
print('Precision:', precision)
print('Recall:', recall)

Epoch 1/10
651/651 [==============================] - 1s 1ms/step - loss: 0.6352 - accuracy: 0.8108
Epoch 2/10
651/651 [==============================] - 1s 1ms/step - loss: 0.4787 - accuracy: 0.8720
Epoch 3/10
651/651 [==============================] - 1s 1ms/step - loss: 0.4357 - accuracy: 0.8868
Epoch 4/10
651/651 [==============================] - 1s 1ms/step - loss: 0.4157 - accuracy: 0.8939
Epoch 5/10
651/651 [==============================] - 1s 1ms/step - loss: 0.4003 - accuracy: 0.8962
Epoch 6/10
651/651 [==============================] - 1s 1ms/step - loss: 0.3899 - accuracy: 0.9004
Epoch 7/10
651/651 [==============================] - 1s 1ms/step - loss: 0.3823 - accuracy: 0.9022
Epoch 8/10
651/651 [==============================] - 1s 1ms/step - loss: 0.3740 - accuracy: 0.9032
Epoch 9/10
651/651 [==============================] - 1s 1ms/step - loss: 0.3650 - accuracy: 0.9045
Epoch 10/10
112/112 [==============================] - 0s 792us/step
90/90 - 0s - loss: 0.4099 - acc